In [1]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [2]:
arquivo_excel = r"C:\Users\thiag\OneDrive\Área de Trabalho\TCC\Scripts\Arquivos Auxiliares\Locais por Data - Noticias.xlsx"

df = pd.read_excel(arquivo_excel)
df = df.rename(columns=lambda x: x.strip().upper())

In [3]:
geolocator = Nominatim(user_agent="geo_goiania")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

LAT_MIN, LAT_MAX = -16.828, -16.546
LON_MIN, LON_MAX = -49.372, -49.043

In [4]:
def geocode_goiania_local(local):
    endereco = f"{local}, Goiânia, GO, Brasil"
    location = geocode(endereco)
    
    if location:
        lat, lon = location.latitude, location.longitude
        if not (LAT_MIN <= lat <= LAT_MAX and LON_MIN <= lon <= LON_MAX):
            lat, lon = -16.6869, -49.2648
    else:
        lat, lon = -16.6869, -49.2648
    return lat, lon

linhas_normalizadas = []

for _, row in df.iterrows():
    data = row['DATA'] if 'DATA' in row else None
    locais = row['LOCAIS'] if 'LOCAIS' in row else ''
    
    for local in str(locais).split(','):
        local = local.strip()
        if local:
            lat, lon = geocode_goiania_local(local)
            linhas_normalizadas.append({'DATA': data, 'LOCAL': local, 'LATITUDE': lat, 'LONGITUDE': lon})

df_normalizado = pd.DataFrame(linhas_normalizadas)

df_normalizado.to_excel(
    r"C:\Users\thiag\OneDrive\Área de Trabalho\TCC\Scripts\Arquivos Processados\Locais_por_Data_GEOCODED.xlsx",
    index=False
)


RateLimiter caught an error, retrying (0/2 tries). Called with (*('T-63, Goiânia, GO, Brasil',), **{}).
Traceback (most recent call last):
  File "c:\Users\thiag\anaconda3\Lib\site-packages\urllib3\connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
  File "c:\Users\thiag\anaconda3\Lib\site-packages\urllib3\connection.py", line 516, in getresponse
    httplib_response = super().getresponse()
  File "c:\Users\thiag\anaconda3\Lib\http\client.py", line 1430, in getresponse
    response.begin()
    ~~~~~~~~~~~~~~^^
  File "c:\Users\thiag\anaconda3\Lib\http\client.py", line 331, in begin
    version, status, reason = self._read_status()
                              ~~~~~~~~~~~~~~~~~^^
  File "c:\Users\thiag\anaconda3\Lib\http\client.py", line 292, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
               ~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^
  File "c:\Users\thiag\anaconda3\Lib\socket.py", line 719, in readinto
    return self._sock.r

In [5]:
import folium

df = pd.read_excel(r"C:\Users\thiag\OneDrive\Área de Trabalho\TCC\Scripts\Arquivos Processados\Locais_por_Data_GEOCODED.xlsx")


mapa = folium.Map(location=[-16.6869, -49.2648], zoom_start=12)  

for _, row in df.iterrows():
    if pd.notna(row['LATITUDE']) and pd.notna(row['LONGITUDE']):
        folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            popup=f"{row['LOCAL']}"
        ).add_to(mapa)


mapa.save("mapa_noticias.html")
